In [1]:
import pandas as pd
import os

# --- 設定項目 ---
# 監査対象の、テキスト抽出済みファイル
INPUT_FILE = '../data/processed/samples_with_text.csv'

# --- データの読み込みと監査 ---
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"--- 監査レポート: '{os.path.basename(INPUT_FILE)}' ---")

    print("\n" + "="*50)
    print("1. データフレームの基本情報 (info)")
    print("="*50)
    # .info() は、総行数、各列のデータ型、nullでない値の数を表示します
    df.info()

    print("\n" + "="*50)
    print("2. 各列の欠損値（null）の数")
    print("="*50)
    # .isnull().sum() は、各列に存在するnull（欠損値）の数を直接カウントします
    print(df.isnull().sum())

except FileNotFoundError:
    print(f"エラー: '{INPUT_FILE}' が見つかりません。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

--- 監査レポート: 'samples_with_text.csv' ---

1. データフレームの基本情報 (info)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   citing_paper_eid        200 non-null    object
 1   citing_paper_doi        200 non-null    object
 2   citing_paper_title      200 non-null    object
 3   cited_data_paper_title  200 non-null    object
 4   abstract                195 non-null    object
 5   full_text               200 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB

2. 各列の欠損値（null）の数
citing_paper_eid          0
citing_paper_doi          0
citing_paper_title        0
cited_data_paper_title    0
abstract                  5
full_text                 0
dtype: int64


In [11]:
import pandas as pd
import os

# --- 設定項目 ---
# 比較元となる、本来200件あるはずのサンプリングリスト
ANNOTATION_LIST_CSV = '../data/ground_truth/annotation_target_list.csv'
# 実際に出力された、テキスト抽出済みのファイル
SAMPLES_WITH_TEXT_CSV = '../data/processed/samples_with_text.csv'

# --- データの読み込み ---
try:
    df_targets = pd.read_csv(ANNOTATION_LIST_CSV)
    df_samples = pd.read_csv(SAMPLES_WITH_TEXT_CSV)
    print("2つのCSVファイルを正常に読み込みました。")

except FileNotFoundError as e:
    print(f"エラー: ファイルが見つかりません。パスを確認してください。 {e}")
    df_targets = pd.DataFrame()
    df_samples = pd.DataFrame()

# --- 照合処理 ---
if not df_targets.empty and not df_samples.empty:
    print("\n" + "="*50)
    print("【2ファイルの照合レポート】")
    print("="*50)

    # 各ファイルのユニークなDOIの数を取得
    target_dois = set(df_targets['citing_paper_doi'].unique())
    sample_dois = set(df_samples['citing_paper_doi'].unique())

    print(f"1. 件数の確認")
    print(f"   - サンプリングリストのユニークDOI数: {len(target_dois)} 件")
    print(f"   - 抽出済みファイルのユニークDOI数:   {len(sample_dois)} 件")

    # --- 差分の特定 ---
    # サンプリングリストには存在するが、抽出済みファイルには存在しないDOIを探す
    missing_dois = target_dois - sample_dois

    print("\n2. 不一致の特定")
    if missing_dois:
        print(f"   - ⚠️ {len(missing_dois)} 件の論文が、最終ファイルから欠落していることが判明しました。")
        
        # 欠落している論文の情報を、元のサンプリングリストから取得して表示
        df_missing = df_targets[df_targets['citing_paper_doi'].isin(missing_dois)]
        
        print("\n--- 欠落していた論文の詳細 ---")
        print(df_missing.to_string())
    else:
        print("   - ✅ ファイル間のDOIに不一致はありませんでした。")
        if len(target_dois) != len(sample_dois):
            print("   - ⚠️ ただし、件数が異なります。重複行などが原因の可能性があります。")

2つのCSVファイルを正常に読み込みました。

【2ファイルの照合レポート】
1. 件数の確認
   - サンプリングリストのユニークDOI数: 200 件
   - 抽出済みファイルのユニークDOI数:   200 件

2. 不一致の特定
   - ✅ ファイル間のDOIに不一致はありませんでした。
